In [13]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [14]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'memory', 'input_width', 'input_size', 'nb_layers', 
                 'sum_activations', 'params']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.pop('input_width')
train_dataset.tail()

,Model_name,input_shape,memory,input_size,nb_layers,sum_activations,params
112,basic_model_32_23,"(1, 32, 32, 3)",2060.0,3072,41,17086005248,17190346
113,basic_model_331_48,"(1, 331, 331, 3)",2033.0,328683,78,65066541656,14519141
114,basic_model_64_21,"(1, 64, 64, 3)",2032.0,12288,38,29858337280,25511850
115,basic_model_112_22,"(1, 112, 112, 3)",2009.0,37632,43,9436995840,15188976
116,basic_model_600_42,"(1, 600, 600, 3)",1995.5,1080000,71,79782663040,12260173


In [38]:
# 1/ Test data :

raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.pop('input_width')
test_dataset.tail()

,Model_name,input_shape,memory,input_size,nb_layers,sum_activations,params
44,ShuffleNet2.0-v2,"(512, 512, 3)",2533.0,786432,208,143166416,14966320
45,ShuffleNet2.0-v2,"(720, 720, 3)",2749.0,1555200,208,284319792,14966320
46,Xception,"(90, 90, 3)",2614.0,24300,134,5636556,22910480
47,Xception,"(512, 512, 3)",2690.0,786432,134,182907144,22910480
48,Xception,"(720, 720, 3)",2750.0,1555200,134,363070504,22910480


In [16]:
Y = train_dataset.pop('memory')
X = train_dataset
Y_f = Y
X_f = X

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [18]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [19]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [20]:
X_f_model_names = X_f.pop('Model_name')
X_f_input_shapes = X_f.pop('input_shape')
dtrain_f = xgb.DMatrix(X_f, label=Y_f)

In [21]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [39]:
Y_e = test_dataset.pop('memory')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [23]:
params = {
    # Parameters that we are going to tune.
    'max_depth':20,
    'min_child_weight': 14,
    'eta':.1,
    'subsample': .8,
    'colsample_bytree': 1.,
    'gamma' : 0.0,
    # Other parameters
    'objective':'reg:squarederror',
}

In [24]:
params['eval_metric'] = "rmse"
num_boost_round = 3000

In [25]:
print(params)

{'max_depth': 20, 'min_child_weight': 14, 'eta': 0.1, 'subsample': 0.8, 'colsample_bytree': 1.0, 'gamma': 0.0, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}


In [26]:
num_boost_round = 40
best_model = xgb.train( 
    params,
    dtrain_f,
    num_boost_round=num_boost_round,
)

In [27]:
#best_model.save_model("model_16-04.model")
loaded_model = best_model

In [40]:
y_pred = loaded_model.predict(dtest_e)
np_test = Y_e.values
y_test = Y_e.values

In [41]:
# Calculate mean absolute percentage error (MAPE)
np_test = y_test
errors = abs(y_pred-np_test)
mape = 100 * (errors / np_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE : ',max(mape))
print('Mape:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE :  28.67935330297649
Mape: 9.04 %.
Accuracy: 90.96 %.


In [42]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [43]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'measured', 'predicted', 'absolute_deviation', 'MAPE'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, np_test[i], y_pred[i],abs(y_pred[i]-np_test[i]), ((abs(y_pred[i]-np_test[i]))/np_test[i])*100])